In [17]:
import json
import yfinance as yf
import pandas as pd
import requests
import os
from datetime import datetime

energy_sector = [
    {"sector": "Oil&Gas",
     "companies": [
         {"name": "Repsol", "country": "Spain", "symbol": "REP.MC" , "data": []},
         {"name": "Exxon Mobil Corporation", "country": "USA", "symbol": "XOM", "data": []}
     ]},
    {"sector": "ElectricPower",
     "companies": [
         {"name": "Iberdrola", "country": "Spain", "symbol": "IBE.MC", "data": []},
         {"name": "NextEra Energy, Inc.", "country": "USA", "symbol": "NEE", "data": []}
     ]},
    {"sector": "RenewableEnergy",
     "companies": [
         {"name": "Acciona", "country": "Spain", "symbol": "ANA.MC", "data": []},
         {"name": "Solaria", "country": "Spain", "symbol": "SOL", "data": []},
         {"name": "Solaredge Technologies Inc.", "country": "USA", "symbol": "SEDG", "data": []}
     ]},
    {"sector": "EnergyServices",
     "companies": [
         {"name": "Técnicas Reunidas", "country": "Spain", "symbol": "TRE.MC", "data": []},
         {"name": "Honeywell International Inc.", "country": "USA", "symbol": "HON", "data": []}
     ]},
    {"sector": "Infrastructure",
     "companies": [
         {"name": "Enagas", "country": "Spain", "symbol": "ENG.MC", "data": []},
         {"name": "Kinder Morgan, Inc.", "country": "USA", "symbol": "KMI", "data": []}
     ]}
]


now_date = datetime.now()       

 
#https://ucdp.uu.se/apidocs/
base_url = "https://ucdpapi.pcr.uu.se/api/gedevents/23.1"

start_date = datetime(2000, 1, 1)  
end_date = datetime(now_date.year , now_date.month , now_date.day )

all_data = []

current_date = start_date
parameters = {
    "pagesize": 10000, 
    "format": "json",        
    "StartDate": start_date.strftime('%Y-%m-%d'),  
    "EndDate": end_date.strftime('%Y-%m-%d') 
}

response = requests.get(base_url, params=parameters)
data = json.loads(response.text)
page = 1
total_pages = data['TotalPages']

next_page = data['NextPageUrl']
if response.status_code == 200:
    all_data.extend(data['Result'])
df = pd.DataFrame(all_data)
while page <total_pages:    
    try:
        if next_page!= "":
            response = requests.get(next_page)    
            data = json.loads(response.text)
            next_page = data['NextPageUrl']
            if response.status_code == 200:                     
                all_data.extend(data['Result'])             
            else:
                print(f"Error al obtener los datos del UCDP para la fecha {current_date}.")
            
            print(f"total_pages: {total_pages} ")    
            page=page+1
            print(print(f"actual page: {page} "))
        else:
            break
        
    except:
        print("Error")
    
df = pd.DataFrame(all_data)   
df.to_csv('ucdp_data_events_2000_2024_01.csv', index=False)

base_url = "https://ucdpapi.pcr.uu.se/api/ucdpprioconflict/23.1"

end_date = datetime(now_date.year , now_date.month , now_date.day )

all_data = []

response = requests.get(base_url, params=parameters)
data = json.loads(response.text)
page = 1
total_pages = data['TotalPages']
print(f"total_pages: {total_pages} ")    

next_page = data['NextPageUrl']
if response.status_code == 200:
    all_data.extend(data['Result'])

while page < total_pages:    
    response = requests.get(next_page)    
    data = json.loads(response.text)
    next_page = data['NextPageUrl']
    if response.status_code == 200:      
       
        all_data.extend(data['Result'])
    else:
        print(f"Error al obtener los datos del UCDP para la fecha {current_date}.")
        
    page=page+1
    print(print(f"actual page: {page} "))

df = pd.DataFrame(all_data)
df.to_csv('ucdp_data_conflicts_2000_2024_01.csv', index=False)   


if not os.path.exists('docs'):
    os.makedirs('docs')

for sector in energy_sector:
    for company in sector['companies']:
        company_data = yf.download(company['symbol'], start='2000-01-01', end=now_date)        
        file_name = f"{company['name']}_data.csv"
        company_data.to_csv(os.path.join('docs', file_name))  
        company['data'] = company_data



KeyboardInterrupt



In [ ]:
import pandas as pd
# Cargar los datos de conflictos y eventos desde los CSV
df_conflictos = pd.read_csv('ucdp_data_conflicts_2000_2024.csv', low_memory=False)
df_eventos = pd.read_csv('ucdp_data_events_2000_2024.csv', low_memory=False)


In [ ]:
df_conflictos.head()

In [ ]:
df_eventos.head()

In [ ]:
columnas_a_eliminar  = df_conflictos.columns.intersection(df_eventos.columns)
columnas_a_eliminar = columnas_a_eliminar.drop(labels=['id','year'])

columnas_a_eliminar

In [ ]:
df_eventos.columns

In [ ]:
df_conflictos.columns

In [36]:

df_eventos = df_eventos.rename(columns={'conflict_dset_id': 'conflict_id'})
# Unir los DataFrames por la columna 'conflict_id'
df_merged = pd.merge(df_conflictos, df_eventos, on='conflict_id', how='inner')

# Verificar el DataFrame resultante
df_merged.columns

Index(['conflict_id', 'location', 'side_a_x', 'side_a_Id', 'side_a_2nd',
       'side_b_x', 'side_b_Id', 'side_b_2nd', 'incompatibility',
       'territory_name', 'year_x', 'intensity_level', 'cumulative_intensity',
       'type_of_conflict', 'start_date', 'start_prec', 'start_date2',
       'start_prec2', 'ep_end', 'ep_end_date', 'ep_end_prec', 'gwno_a',
       'gwno_a_2nd', 'gwno_b', 'gwno_b_2nd', 'gwno_loc', 'region_x', 'version',
       'id', 'relid', 'year_y', 'active_year', 'code_status',
       'type_of_violence', 'conflict_new_id', 'conflict_name', 'dyad_dset_id',
       'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id',
       'side_a_y', 'side_b_dset_id', 'side_b_new_id', 'side_b_y',
       'number_of_sources', 'source_article', 'source_office', 'source_date',
       'source_headline', 'source_original', 'where_prec', 'where_coordinates',
       'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude',
       'geom_wkt', 'priogrid_gid', 'country', 'country_i

In [18]:
df_merged.to_csv('ucdp_data_conflicts_and_events_2000_2024_FULL.csv', index=False)   

In [19]:
df_merged.head()

,conflict_id,location,side_a_x,side_a_Id,side_a_2nd,side_b_x,side_b_Id,side_b_2nd,incompatibility,territory_name,...,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob
0,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,2009-08-09T00:00:00,4,10,0,0,14,14,14,NaN,NaN
1,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,2009-11-15T00:00:00,1,0,0,0,1,1,1,NaN,NaN
2,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,2009-07-04T00:00:00,0,0,0,16,16,16,16,NaN,NaN
3,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,2009-07-14T00:00:00,3,15,0,0,18,18,18,NaN,NaN
4,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,2009-10-03T00:00:00,1,0,0,0,1,1,1,NaN,NaN


In [30]:
# conflict_id_counts = df_eventos['conflict_id'].value_counts()

# Crear un diccionario con los conteos de conflict_id
# conflict_id_counts_dict = conflict_id_counts.to_dict()

# Agregar una nueva columna al DataFrame df_merged con los conteos de conflict_id
# df_merged['number_of_events'] = df_merged['conflict_id'].map(conflict_id_counts_dict)
df_merged.head()

,conflict_id,location,side_a_x,side_a_Id,side_a_2nd,side_b_x,side_b_Id,side_b_2nd,incompatibility,territory_name,...,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob,conflict_id_counts,number_of_events
0,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,10,0,0,14,14,14,NaN,NaN,168,84
1,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,0,0,0,1,1,1,NaN,NaN,168,84
2,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,0,0,16,16,16,16,NaN,NaN,168,84
3,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,15,0,0,18,18,18,NaN,NaN,168,84
4,11342,India,Government of India,141,NaN,GNLA,1163,NaN,1,Garoland,...,0,0,0,1,1,1,NaN,NaN,168,84


In [42]:
selected_columns = df_merged[['conflict_id','conflict_name', 'date_start', 'date_end', 'best', 'deaths_civilians','deaths_unknown','type_of_violence','latitude', 'longitude', 'country', 'region_x']]
selected_columns.head()

,conflict_id,conflict_name,date_start,date_end,best,deaths_civilians,deaths_unknown,type_of_violence,latitude,longitude,country,region_x
0,11342,Lashkar of Mohmand tribe - TTP,2009-08-09T00:00:00,2009-08-09T00:00:00,14,0,0,2,34.463860,71.351161,Pakistan,3
1,11342,Lashkar of Mohmand tribe - TTP,2009-11-15T00:00:00,2009-11-15T00:00:00,1,0,0,2,34.814722,71.421667,Pakistan,3
2,11342,Lashkar of Mohmand tribe - TTP,2009-07-04T00:00:00,2009-07-04T00:00:00,16,0,16,2,34.463860,71.351161,Pakistan,3
3,11342,Lashkar of Mohmand tribe - TTP,2009-07-14T00:00:00,2009-07-14T00:00:00,18,0,0,2,34.594755,71.455747,Pakistan,3
4,11342,Lashkar of Mohmand tribe - TTP,2009-10-03T00:00:00,2009-10-03T00:00:00,1,0,0,2,34.800000,71.466667,Pakistan,3


In [45]:
#grouped_conflicts = selected_columns.agg({
#    'date_start': 'min', 
#    'date_end': 'max',     # Selecciona la fecha de fin más tardía
#    'best': 'min',         # Suma la gravedad total del conflicto
#    'deaths_civilians': 'sum',  # Suma el número total de civiles muertos
#    'deaths_unknown': 'sum',     # Suma el número total de muertes de origen desconocido
#    'type_of_violence': 'first',  # Toma el primer valor de tipo de violencia (puede ser arbitrario)
#    'latitude': 'mean',    # Calcula la media de latitud
#    'longitude': 'mean',   # Calcula la media de longitud
#    'country': 'first',    # Toma el primer valor de país (puede ser arbitrario)
#    'region_x': 'first'      # Toma el primer valor de región (puede ser arbitrario)
#}).reset_index()

selected_columns = selected_columns.sort_values(by='date_start')
selected_columns.to_csv('docs/conflicts_events_FULL_filtered.csv', index=False)
selected_columns.head()

,conflict_id,conflict_name,date_start,date_end,best,deaths_civilians,deaths_unknown,type_of_violence,latitude,longitude,country,region_x
3027913,329,Ethiopia: Ogaden,2000-01-01T00:00:00,2000-01-23T00:00:00,0,0,0,1,7.000000,44.000000,Ethiopia,4
3016205,327,Angola: Government,2000-01-01T00:00:00,2000-11-10T00:00:00,0,0,0,1,-13.000000,20.500000,Angola,4
5158653,364,India: Kashmir,2000-01-01T00:00:00,2000-10-15T00:00:00,153,0,153,1,33.916670,76.666670,India,3
5256659,386,Algeria: Government,2000-01-01T00:00:00,2000-01-01T00:00:00,2,0,0,1,36.855833,3.866667,Algeria,4
4866206,352,Chechen Republic of Ichkeria - Civilians,2000-01-01T00:00:00,2000-01-11T00:00:00,0,0,0,3,43.305800,45.747700,Russia (Soviet Union),3


In [44]:
df_merged.describe()

,conflict_id,incompatibility,year_x,intensity_level,cumulative_intensity,type_of_conflict,start_prec,start_prec2,ep_end,ep_end_prec,...,event_clarity,date_prec,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnob
count,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,0.0,...,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,5.517264e+06,27725.000000
mean,6.712921e+02,1.750931e+00,1.999114e+03,1.525420e+00,8.890771e-01,3.378281e+00,2.923012e+00,1.765493e+00,6.694278e-02,NaN,...,1.090623e+00,1.191328e+00,1.445484e+00,2.761214e+00,1.140080e+00,9.150322e-01,6.261810e+00,7.910367e+00,5.924867e+00,699.863192
std,2.155841e+03,4.345621e-01,1.781678e+01,4.993534e-01,3.140367e-01,4.962111e-01,2.287665e+00,1.452775e+00,2.499229e-01,NaN,...,2.870717e-01,5.126483e-01,4.286608e+01,1.992026e+01,8.333316e+00,1.543954e+02,1.638962e+02,1.697508e+02,1.638417e+02,149.442258
min,2.050000e+02,1.000000e+00,1.946000e+03,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,NaN,...,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,369.000000
25%,2.890000e+02,2.000000e+00,1.986000e+03,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,NaN,...,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,770.000000
50%,3.090000e+02,2.000000e+00,2.003000e+03,2.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,NaN,...,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,770.000000
75%,3.330000e+02,2.000000e+00,2.014000e+03,2.000000e+00,1.000000e+00,4.000000e+00,6.000000e+00,1.000000e+00,0.000000e+00,NaN,...,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,6.000000e+00,5.000000e+00,770.000000
max,1.543800e+04,3.000000e+00,2.022000e+03,2.000000e+00,1.000000e+00,4.000000e+00,6.000000e+00,6.000000e+00,1.000000e+00,NaN,...,2.000000e+00,5.000000e+00,1.416200e+04,6.916000e+03,5.964000e+03,7.534000e+04,7.534000e+04,7.425600e+04,7.548200e+04,800.000000


In [35]:
df_eventos[df_eventos["conflict_id"]==364].count()

id                   5800
relid                5800
year                 5800
active_year          5800
code_status          5800
type_of_violence     5800
conflict_id          5800
conflict_new_id      5800
conflict_name        5800
dyad_dset_id         5800
dyad_new_id          5800
dyad_name            5800
side_a_dset_id       5800
side_a_new_id        5800
side_a               5800
side_b_dset_id       5800
side_b_new_id        5800
side_b               5800
number_of_sources    5800
source_article       5800
source_office        3649
source_date          3649
source_headline      3649
source_original      5346
where_prec           5800
where_coordinates    5800
where_description    5715
adm_1                5768
adm_2                5522
latitude             5800
longitude            5800
geom_wkt             5800
priogrid_gid         5800
country              5800
country_id           5800
region               5800
event_clarity        5800
date_prec            5800
date_start  